<a href="https://colab.research.google.com/github/carlosinator/cil-sentiment/blob/main/infer_submission.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import auth
auth.authenticate_user()

In [2]:
%%bash
pip3 install transformers emoji==0.6.0 keras_nlp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 25.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 6.4 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 576.5/576.5 kB 39.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 30.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 94.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 57.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 753.1/753.1 kB 64.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 111.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 524.1/524.1 MB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 54.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 115.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━

In [3]:
!git clone https://github.com/carlosinator/cil-sentiment.git

Cloning into 'cil-sentiment'...
remote: Enumerating objects: 110, done.
remote: Counting objects: 100% (110/110), done.
remote: Compressing objects: 100% (99/99), done.
remote: Total 110 (delta 51), reused 17 (delta 4), pack-reused 0
Receiving objects: 100% (110/110), 158.12 KiB | 5.45 MiB/s, done.
Resolving deltas: 100% (51/51), done.


In [4]:
import tensorflow as tf
import tensorflow_probability as tfp
import keras_nlp
import numpy as np
import pandas as pd
import sklearn
from sklearn import metrics
import transformers
from transformers import AutoTokenizer, TFAutoModel, AutoConfig, TFAutoModelForSequenceClassification
import matplotlib.pyplot as plt
import pickle
import re
import subprocess as sp
import os
from threading import Thread , Timer
import sched, time

import sys
sys.path.append("./cil-sentiment/models")
sys.path.append("./cil-sentiment/")
from gru_models import GRUModel, VGRUModel
import utils

# reproducibility
transformers.set_seed(0) # sets the seed in random, numpy, and tf

Using TensorFlow backend


## Copy Test Data

In [5]:
# copy preprocessed data from google cloud storage
!gsutil cp "gs://cil_2023/test_preprocessed.txt" .

Copying gs://cil_2023/test_preprocessed.txt...
/ [1 files][798.3 KiB/798.3 KiB]                                                
Operation completed over 1 objects/798.3 KiB.                                    


## Parse Test Data into df

In [6]:
# Read the file line by line, split on the first comma only, and append to a list
data = []
with open('test_preprocessed.txt', 'r') as file:
    for line in file:
        split_line = line.strip().split(',', 1)  # split only on the first comma
        if len(split_line) == 2:  # to ensure there are no lines with missing values
            data.append(split_line)

# Create a DataFrame from the list
df = pd.DataFrame(data, columns=['id', 'tweet'])

## Tokenize Dataset

In [7]:
# tokenize data set
model_name = "vinai/bertweet-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenized_tweets = tokenizer.batch_encode_plus(df['tweet'].tolist(),
                                    padding=True, truncation=True,
                                    return_tensors='tf')



Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [8]:
# some variable definitions
USE_MODEL = "basemodel"
run_name = "inference_" + USE_MODEL + "_fullmodel"

In [9]:
# download the model
run_name = "inference_basemodel_fullmodel"
!gsutil cp -r {"gs://cil_2023/models/" + run_name} .

Copying gs://cil_2023/models/inference_basemodel_fullmodel/fingerprint.pb...
Copying gs://cil_2023/models/inference_basemodel_fullmodel/keras_metadata.pb...
Copying gs://cil_2023/models/inference_basemodel_fullmodel/saved_model.pb...
Copying gs://cil_2023/models/inference_basemodel_fullmodel/variables/variables.data-00000-of-00001...
==> NOTE: You are downloading one or more large file(s), which would
run significantly faster if you enabled sliced object downloads. This
feature is enabled by default but requires that compiled crcmod be
installed (see "gsutil help crcmod").

|
==> NOTE: You are performing a sequence of gsutil operations that may
run significantly faster if you instead use gsutil -m cp ... Please
see the -m section under "gsutil help options" for further information
about when gsutil -m can be advantageous.

Copying gs://cil_2023/models/inference_basemodel_fullmodel/variables/variables.index...
| [5 files][  1.5 GiB/  1.5 GiB]   70.7 MiB/s                                

In [10]:
# Load the saved model
model = tf.keras.models.load_model(run_name)

In [ ]:
# Perform the inference
probabilities = model.predict(dict(tokenized_tweets))

In [18]:
if USE_MODEL == "basemodel":
  # on base model, we will get logits, so apply a softmax
  probabilities = tf.nn.softmax(probabilities["logits"])

# Convert probabilities into class predictions (0 or 1)
# and then map 0 to 1 and 1 to -1 (that is Leo's great mapping:))
predictions = np.argmax(probabilities, axis=-1)
predictions = np.where(predictions == 0, 1, -1)

# Write the results to a text file
with open('predictions.csv', 'w') as file:
    file.write("Id,Prediction\n")
    for i, prediction in enumerate(predictions):
        file.write(f"{df['id'].iloc[i]},{prediction}\n")